<a href="https://colab.research.google.com/github/Kennenzz/predict-employee-attrition/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**XÂY DỰNG MÔ HÌNH DỰ ĐOÁN NGUY CƠ NGHỈ VIỆC CỦA NHÂN VIÊN**
---

**#Import các thư viện sẽ sử dụng trong đồ án bao gồm:**
*   **Pandas**
*   **Numpy**
*   **Matplotlib**
*   **Seaborn**
*   **Matplotlib**
*   **Imblearn**
*   **Sklearn**
*   **LightGBM**
*   **XGBoost**
*   **Pytorch Tabnet**

In [ ]:
import pandas as pd
import joblib
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from pytorch_tabnet.tab_model import TabNetClassifier
import lightgbm as lgb
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

---
**Đọc file dataset**

In [ ]:
try:
    df = pd.read_csv("Dataset.csv")
    display(df.head())
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file dataset trong dữ liệu folder")
except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

---
**Mô tả tổng quan về dataset**

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

**Tổng quan: Dataset thông tin của nhân viên gồm có 1470 mẫu và 35 cột (thuộc tinh)**


---
# Phân loại các biến trong dataset

In [ ]:
numerical_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

print("Biến số:")
display(numerical_cols)

print("\nBiến phân loại:")
display(categorical_cols)

---
**Tiền xử lý dữ liệu**

In [ ]:
# Xử lý chuẩn hóa dữ liệu
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Chuyển đổi sang kiểu số, lỗi sẽ thành NaN
    if df[col].isnull().any():
        print(f"Cảnh báo: Cột '{col}' có giá trị không thể chuyển đổi sang số.")
    print(f"Cột '{col}' đã được chuyển đổi sang kiểu số.")


for col in categorical_cols:
    unique_values = df[col].unique()
    print(f"Giá trị duy nhất trong cột '{col}': {unique_values}")
    df[col] = df[col].str.strip().str.lower()   # Loại bỏ khoảng trắng và chuyển về chữ thường  
    unique_values_after = df[col].unique()
    print(f"Giá trị duy nhất trong cột '{col}' sau khi xử lý: {unique_values_after}")

Xử lý dữ liệu thiếu

In [ ]:
df.isnull().sum()


*Không có dữ liệu thiếu trong dataset*

*Xử lý dữ liệu dư thừa, dữ liệu gây nhiễu (không liên quan)*

In [ ]:
df.drop(columns=['Over18', 'EmployeeNumber', 'EmployeeCount','StandardHours'], axis =1, inplace= True)
# Cập nhật lại danh sách biến phân loại sau khi loại bỏ cột
numerical_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

Qua quan sát dữ liệu thì có thể thấy các cột dữ liệu không cần thiết như sau:
*   **Over18** tất cả nhân viên đều đã trên 18 tuổi
*   **EmployeeCount** luôn có giá trị là 1
*   **EmployeeNumber** luôn có giá trị khác nhau như việc nhân viên có mã nhân viên
*   **StandardHours** luôn có giá trị là 80

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

**Kiểu tra xem có dữ liệu trùng lặp hay không**

In [ ]:

duplicate_rows = df[df.duplicated()]
print(f"Số hàng trùng lặp: {duplicate_rows.shape[0]}")
print("Các hàng trùng lặp:")
display(duplicate_rows)


**Xử lý mã hóa biến phân loại**

In [ ]:
print("Các biến phân loại sau khi loại bỏ cột không cần thiết:")
display(categorical_cols)


*Mã hóa biến mục tiêu "Attrition" thành 0/1 và xử lý mã hóa các biến phân loại*

In [ ]:
label_encoder = LabelEncoder()
df['Attrition'] = label_encoder.fit_transform(df['Attrition'])

def mahoabienphanloai(encoder):
    for col in categorical_cols:
        if col not in ['Attrition','Over18']:  # Bỏ qua cột đã mã hóa hoặc không cần mã hóa
            df[col] = encoder.fit_transform(df[col])
mahoabienphanloai(label_encoder)

In [ ]:
display(df.info())
# hiện 100 dòng đầu tiên
display(df.head(100))

**Dùng IQR phát hiện các outliers và tạo ra bộ dữ liệu loại bỏ các outlier**

In [ ]:
# Phát hiện ngoại lệ sử dụng IQR
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1
print("IQR values:")
print(IQR)
outlier_condition = ((df[numerical_cols] < (Q1 - 1.5 * IQR)) | (df[numerical_cols] > (Q3 + 1.5 * IQR)))
outlier_indices = outlier_condition.any(axis=1)
print("Chỉ số của các hàng có ngoại lệ:")
print(outlier_indices[outlier_indices].index.tolist())
print(f"\nSố lượng ngoại lệ phát hiện: {outlier_indices.sum()}")
df_cleaned_outliers = df[~outlier_indices]
print(f"Số lượng mẫu ban đầu: {df.shape[0]}")
print(f"Số lượng mẫu sau khi loại bỏ ngoại lệ: {df_cleaned_outliers.shape[0]}")

Do việc dùng IQR để loại bỏ các ngoại lệ trong dataset gây ra sự giảm sút lớn về mặt dữ liệu mẫu (loại bỏ đến 50% mẫu) nên sẽ dùng thêm phương pháp cắt tỉa winsorizing đặt các giá trị ngoại lai về 1 ngưỡng chứ không xóa bỏ hoàn toàn giá trị ngoại lai

In [ ]:
# Dùng phương pháp winsorizing để xử lý ngoại lệ
from scipy.stats.mstats import winsorize
for col in numerical_cols:
    df[col] = winsorize(df[col], limits=[0.05, 0.05])
    # Cắt tỉa 5% ở hai đầu
    # Giữ lại 90% dữ liệu ở giữa, thay thế 5% giá trị nhỏ nhất và lớn nhất bằng ngưỡng tương ứng
    print(f"Cột '{col}' đã được xử lý ngoại lệ bằng phương pháp winsorizing.")

In [ ]:
# Phân tích và tìm ra các đặc trưng quan trọng trong bộ dữ liệu
X = df.drop('Attrition', axis=1)
y = df['Attrition']
shap_explainer = shap.TreeExplainer(XGBClassifier().fit(X, y))
shap_values = shap_explainer.shap_values(X)
shap.summary_plot(shap_values, X)
important_features = np.abs(shap_values).mean(axis=0)
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': important_features})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
display(feature_importance)
# lưu lại 12 đặc trưng quan trọng nhất trong bộ dữ liệu khác
top_12_features = feature_importance['Feature'].head(12).tolist()
df_top_12 = df[top_12_features + ['Attrition']]
display(df_top_12.head())
df_top_12_cleaned_outliers = df_cleaned_outliers[top_12_features + ['Attrition']]
display(df_top_12_cleaned_outliers.head())


---
#**KHAI PHÁ DỮ LIỆU EDA**


*Tổng quan về tình hình nghỉ việc của nhân viên*

In [ ]:
# 1. Đếm số lượng nhân viên nghỉ việc và không nghỉ việc
attrition_counts = df['Attrition'].value_counts()
print("Số lượng nhân viên nghỉ việc và không nghỉ việc:")
display(attrition_counts)

# 2. Vẽ biểu đồ cột cho tỷ lệ nghỉ việc
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Attrition', palette='viridis')
plt.title('Tỷ lệ nghỉ việc vs ở lại')
plt.xticks([0, 1], ['Ở lại', 'Nghỉ việc'])
plt.xlabel('Nghỉ việc')
plt.ylabel('Số lượng nhân viên')
plt.show()

# 3. Hiển thị phần trăm nhân viên nghỉ việc
if 1 in attrition_counts.index:
	attrition_percentage = (attrition_counts[1] / len(df)) * 100
	print(f"\nPhần trăm nhân viên nghỉ việc: {attrition_percentage:.2f}%")
else:
	print("\nKhông tìm thấy giá trị '1' trong cột 'Attrition'. Kiểm tra lại dữ liệu mã hóa.")

In [ ]:
# Lấy các biến số (loại trừ biến kiểu bool)
filtered_numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
# Tính số hàng và cột cho subplot
num_plots = len(filtered_numerical_cols)
cols = 5
rows = math.ceil(num_plots / cols)

plt.figure(figsize=(24, 18))
for idx, col in enumerate(filtered_numerical_cols):
    plt.subplot(rows, cols, idx + 1)
    sns.histplot(df[col], bins=20, kde=True, color='skyblue')
    plt.title(f'Biểu đồ Histogram: {col}')
plt.tight_layout()
plt.show()

# Vẽ countplot cho các biến phân loại (chỉ vẽ cho các biến đã được mã hóa nhị phân)
plt.figure(figsize=(24, 12))
for i, col in enumerate(df.columns[df.dtypes == 'bool']):
    plt.subplot(4, 5, i + 1)
    sns.countplot(x=col, data=df, palette='viridis')
    plt.title(f'Biểu đồ Countplot: {col}')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(24, 18))
for idx, col in enumerate(filtered_numerical_cols):
    plt.subplot(rows, cols, idx + 1)
    sns.histplot(data=df, x=col, hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
    plt.title(f'Histogram: {col} theo Attrition')
plt.tight_layout()
plt.show()

In [ ]:
# Vẽ countplot giữa các biến phân loại và biến mục tiêu Attrition
plt.figure(figsize=(24, 18))
for idx, col in enumerate(categorical_cols):
    plt.subplot(3, 3, idx + 1)
    # Nếu cột đã được one-hot encoding, vẽ countplot cho từng biến mới liên quan
    related_cols = [c for c in df.columns if c.startswith(col + "_") or c == col]
    for rel_col in related_cols:
        sns.countplot(data=df, x=rel_col, hue='Attrition', palette='viridis')
        plt.title(f'Countplot: {rel_col} vs Attrition')
        plt.xlabel(rel_col)
        plt.ylabel('Số lượng')
    plt.title(f'Countplot: {col} vs Attrition')
    plt.xlabel(col)
    plt.ylabel('Số lượng')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

# Histogram for Age
plt.subplot(1, 2, 1)
sns.histplot(data=df, x='Age', hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
plt.title('Phân phối Age theo Attrition')
plt.xlabel('Age')
plt.ylabel('Số lượng')

# Histogram for MonthlyIncome
plt.subplot(1, 2, 2)
sns.histplot(data=df, x='MonthlyIncome', hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
plt.title('Phân phối MonthlyIncome theo Attrition')
plt.xlabel('MonthlyIncome')
plt.ylabel('Số lượng')

plt.tight_layout()
plt.show()

In [ ]:
# Tính ma trận tương quan
corr_matrix = df.corr()

# Vẽ heatmap với seaborn
plt.figure(figsize=(18, 14))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap Ma trận tương quan các biến')
plt.show()

In [ ]:

print("Phân phối của biến mục tiêu 'Attrition':")
print(df['Attrition'].value_counts(normalize=True))
print(df['Attrition'].value_counts())

*Chia bộ dữ liệu ra làm 2 phần 70/30 để huấn luyện mô hình*

In [ ]:
# chia bộ dữ liệu ra làm 2 phần 70/30 để huấn luyện mô hình
X = df.drop('Attrition', axis=1)
y = df['Attrition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Số lượng mẫu trong tập huấn luyện: {X_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra: {X_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp:\n{y_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp:\n{y_test.value_counts()}")

# chia bộ dữ liệu đã loại bỏ outliers ra làm 2 phần 70/30
X_cleaned_outliers = df_cleaned_outliers.drop('Attrition', axis=1)
y_cleaned_outliers = df_cleaned_outliers['Attrition']
X_cleaned_outliers_train, X_cleaned_outliers_test, y_cleaned_outliers_train, y_cleaned_outliers_test = train_test_split(
	X_cleaned_outliers, y_cleaned_outliers, test_size=0.3, random_state=42, stratify=y_cleaned_outliers
)
print(f"Số lượng mẫu trong tập huấn luyện (sau khi resample và loại bỏ ngoại lệ): {X_cleaned_outliers_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra (sau khi resample và loại bỏ ngoại lệ): {X_cleaned_outliers_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp (sau khi resample và loại bỏ ngoại lệ):\n{y_cleaned_outliers_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp (sau khi resample và loại bỏ ngoại lệ):\n{y_cleaned_outliers_test.value_counts()}")

# Chia bộ dữ liệu chỉ có 12 đặc trưng quan trọng nhất ra làm 2 phần 70/30
X_top_12 = df_top_12.drop('Attrition', axis=1)
y_top_12 = df_top_12['Attrition']
X_top12_train, X_top12_test, y_top12_train, y_top12_test = train_test_split(X_top_12, y_top_12, test_size=0.3, random_state=42, stratify=y_top_12)
print(f"Số lượng mẫu trong tập huấn luyện (chỉ 12 đặc trưng quan trọng nhất): {X_top12_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra (chỉ 12 đặc trưng quan trọng nhất): {X_top12_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp (chỉ 12 đặc trưng quan trọng nhất):\n{y_top12_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp (chỉ 12 đặc trưng quan trọng nhất):\n{y_top12_test.value_counts()}")


Thực hiện Feature Scaling tập X để chuẩn hóa dữ liệu

In [ ]:
# Thực hiện Feature Scaling tập X để chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)     
X_cleaned_outliers_train = scaler.transform(X_cleaned_outliers_train)
X_cleaned_outliers_test = scaler.transform(X_cleaned_outliers_test)
X_top_12_train = scaler.fit_transform(X_top_12_train)
X_top_12_test = scaler.transform(X_top_12_test)

**Xử lý mất cân bằng dữ liệu dùng phương pháp SMOTE**

In [ ]:
# Xử lý mất cân bằng dữ liệu sử dụng SMOTE trên bộ dữ liệu huấn luyện
smote = SMOTE(random_state=42)
X_resampled_train, y_resampled_train = smote.fit_resample(X_train, y_train)
print(f"Số lượng mẫu trong tập huấn luyện sau khi resample: {X_resampled_train.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp sau khi resample:\n{y_resampled_train.value_counts()}")
# Xử lý mất cân bằng dữ liệu sử dụng SMOTE trên bộ dữ liệu đã loại bỏ outliers
smote_outliers = SMOTE(random_state=42)
X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train = smote_outliers.fit_resample(X_cleaned_outliers_train, y_cleaned_outliers_train)
print(f"Số lượng mẫu trong tập huấn luyện (sau khi loại bỏ outliers) sau khi resample: {X_resampled_cleaned_outliers_train.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện (sau khi loại bỏ outliers) theo lớp sau khi resample:\n{y_resampled_cleaned_outliers_train.value_counts()}")
# Xử lý mất cân bằng dữ liệu sử dụng SMOTE trên bộ dữ liệu chỉ có 12 đặc trưng quan trọng nhất
smote_top_12 = SMOTE(random_state=42)
X_resampled_top_12_train, y_resampled_top_12_train = smote_top_12.fit_resample(X_top_12_train, y_top_12_train)
print(f"Số lượng mẫu trong tập huấn luyện (chỉ 12 đặc trưng quan trọng nhất) sau khi resample: {X_resampled_top_12_train.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện (chỉ 12 đặc trưng quan trọng nhất) theo lớp sau khi resample:\n{y_resampled_top_12_train.value_counts()}")


# Xây dựng mô hình (Build Model)

Xây dựng mô hình với bộ dữ liệu chưa qua resample bằng phương pháp SMOTE

In [ ]:
# Huấn luyện mô hình Logistic Regression và đánh giá
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Báo cáo phân loại:") 
print(classification_report(y_test, y_pred))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred):.4f}")

In [ ]:
# Huấn luyện mô hình Decision Tree và đánh giá
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)

y_pred_dt = model_dt.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_dt))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_dt), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Decision Tree')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_dt):.4f}")

In [ ]:
#huấn luyện mô hình random forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_rf))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_rf), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Random Forest')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_rf):.4f}")

In [ ]:
#huấn luyện mô hình KNN (K-Nearest Neighbors) với điểm k chạy từ 1 tới 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_train, y_train)
    y_pred_knn = model_knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_knn))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_knn), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - KNN')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_knn):.4f}")



In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu chỉ qua tiền xử lý 
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_xgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_xgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - XGBoost')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM và đánh giá
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_lgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_lgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - LightGBM')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_lgb):.4f}")


In [ ]:
# Huấn luyện mô hình SVMmodel_svm = SVC(random_state=42)
model_svm = SVC(random_state=42)
model_svm.fit(X_train, y_train)
y_pred_svm = model_svm.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_svm))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_svm), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - SVM')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_svm):.4f}")


In [ ]:
# Huấn luyện mô hình Tabnet
# TabNetClassifier không nhận 'random_state' — dùng 'seed' nếu cần khởi tạo ngẫu nhiên
# X_train / y_train có thể đã là numpy.ndarray (sau scaler), do đó tránh gọi .values trên numpy arrays
X_tabnet = X_train if isinstance(X_train, np.ndarray) else X_train.values
y_tabnet = y_train if isinstance(y_train, np.ndarray) else y_train.values

model_tabnet = TabNetClassifier(seed=42)
model_tabnet.fit(X_tabnet, y_tabnet, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)

X_test_tabnet = X_test if isinstance(X_test, np.ndarray) else X_test.values
y_pred_tabnet = model_tabnet.predict(X_test_tabnet)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_tabnet))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_tabnet), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - TabNet')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_tabnet):.4f}")


In [ ]:
# Dùng SHAP để diễn giải tất cả các mô hình đã huấn luyện
explainer_lr = shap.Explainer(model.predict, X_train)
shap_values_lr = explainer_lr(X_test)
shap.summary_plot(shap_values_lr, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - Logistic Regression")
plt.show()
explainer_dt = shap.Explainer(model_dt.predict, X_train)
shap_values_dt = explainer_dt(X_test)
shap.summary_plot(shap_values_dt, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - Decision Tree")
plt.show()
explainer_rf = shap.Explainer(model_rf.predict, X_train)
shap_values_rf = explainer_rf(X_test)
shap.summary_plot(shap_values_rf, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - Random Forest")
plt.show()
explainer_knn = shap.Explainer(model_knn.predict, X_train)
shap_values_knn = explainer_knn(X_test)
shap.summary_plot(shap_values_knn, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - KNN")
plt.show()
explainer_xgb = shap.Explainer(model_xgb.predict, X_train)
shap_values_xgb = explainer_xgb(X_test)
shap.summary_plot(shap_values_xgb, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - XGBoost")
plt.show()
explainer_lgb = shap.Explainer(model_lgb.predict, X_train)
shap_values_lgb = explainer_lgb(X_test)
shap.summary_plot(shap_values_lgb, X_test, feature_names=X.columns, plot_type="bar", show=False)
plt.title("SHAP Summary Plot - LightGBM")
plt.show()
try:
	explainer_svm = shap.Explainer(model_svm.predict, X_train)
	shap_values_svm = explainer_svm(X_test)
	if isinstance(shap_values_svm, list):
		vals = shap_values_svm[1]  # Lấy giá trị SHAP cho lớp dương
	else:
		vals = shap_values_svm
	shap.summary_plot(vals, X_test, plot_type="bar", feature_names=X_test.columns, show=False)
	plt.title("Tầm quan trọng của các đặc trưng - SVM")
	plt.show()
except Exception as e:
	print("Không thể tạo SHAP cho SVM:", e)
try:
	explainer_tabnet = shap.Explainer(model_tabnet.predict, X_tabnet)
	shap_values_tabnet = explainer_tabnet(X_test_tabnet)
	shap.summary_plot(shap_values_tabnet, X_test_tabnet, feature_names=X.columns, plot_type="bar", show=False)
	plt.title("SHAP Summary Plot - TabNet")
	plt.show()
except Exception as e:
	print("Không thể tạo SHAP cho TabNet:", e)

In [ ]:
# Dùng tensorflow để so sánh các mô hình đã huấn luyện
models = {
    "Logistic Regression": model,
    "Decision Tree": model_dt,
    "Random Forest": model_rf,
    "KNN": model_knn,
    "XGBoost": model_xgb,
    "LightGBM": model_lgb,
    "SVM": model_svm,
    "TabNet": model_tabnet
}

def _to_array(X):
    # Nếu là pandas DataFrame/Series thì dùng .values, nếu đã là numpy.ndarray thì trả về trực tiếp
    return X if isinstance(X, np.ndarray) else getattr(X, "values", X)

auc_scores = {}
for name, mdl in models.items():
    try:
        X_input = _to_array(X_test)
        if name == "TabNet":
            # TabNet expects numpy input; X_input đảm bảo là numpy.ndarray
            y_probs = mdl.predict_proba(X_input)[:, 1]
        else:
            if hasattr(mdl, "predict_proba"):
                y_probs = mdl.predict_proba(X_input)[:, 1]
            elif hasattr(mdl, "decision_function"):
                y_probs = mdl.decision_function(X_input)
            else:
                # fallback to hard predictions
                y_probs = mdl.predict(X_input)
        auc = roc_auc_score(y_test, y_probs)
        auc_scores[name] = auc
    except Exception as e:
        # Nếu có lỗi, in thông báo và gán NaN cho model đó
        print(f"Không thể tính AUC cho {name}: {e}")
        auc_scores[name] = np.nan

# Vẽ biểu đồ AUC cho các mô hình
plt.figure(figsize=(10, 6))
model_names = list(auc_scores.keys())
auc_values = [auc_scores[m] for m in model_names]
sns.barplot(x=model_names, y=auc_values, palette='viridis')
plt.xticks(rotation=45)
plt.ylabel('AUC Score')
plt.title('So sánh AUC của các mô hình')
plt.ylim(0, 1)
plt.show()

# Vẽ ROC Curve cho các mô hình
plt.figure(figsize=(10, 6))
for name, mdl in models.items():
    try:
        X_input = _to_array(X_test)
        if name == "TabNet":
            y_probs = mdl.predict_proba(X_input)[:, 1]
        else:
            if hasattr(mdl, "predict_proba"):
                y_probs = mdl.predict_proba(X_input)[:, 1]
            elif hasattr(mdl, "decision_function"):
                y_probs = mdl.decision_function(X_input)
            else:
                y_probs = mdl.predict(X_input)
        fpr, tpr, _ = roc_curve(y_test, y_probs)
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_scores[name]:.2f})')
    except Exception as e:
        print(f"Không thể vẽ ROC Curve cho {name}: {e}")
plt.plot([0, 1], [0, 1], 'k--')  # Đường chéo
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve của các mô hình')
plt.legend(loc='lower right')
plt.show()



Xây dựng mô hình với bộ dữ liệu đã qua resample bằng phương pháp SMOTE

In [ ]:
#Dùng data đã qua resample để train model Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_resampled_train, y_resampled_train)
y_pred = model.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Logistic Regression (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred):.4f}")

In [ ]:
# Huấn luyện mô hình Decision Tree và đánh giá với dữ liệu đã qua resample
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_resampled_train, y_resampled_train)

y_pred_dt = model_dt.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_dt))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_dt), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Decision Tree (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_dt):.4f}")

In [ ]:
# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu đã qua resample
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_resampled_train, y_resampled_train)
y_pred_rf = model_rf.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_rf))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_rf), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - RandomForest (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_rf):.4f}")

In [ ]:
# Huấn luyện mô hình KNN với dữ liệu đã qua resample cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_resampled_train, y_resampled_train)
    y_pred_knn = model_knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_resampled_train, y_resampled_train)
y_pred_knn = model_knn.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_knn))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_knn), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - KNN (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_knn):.4f}")

In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu đã qua resample
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_resampled_train, y_resampled_train)
y_pred_xgb = model_xgb.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_xgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_xgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - XGBoost (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM với dữ liệu đã qua resample
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_resampled_train, y_resampled_train)
y_pred_lgb = model_lgb.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_lgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_lgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - LightGBM (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_lgb):.4f}")

In [ ]:
# Huấn luyện mô hình SVM với dữ liệu đã qua resample
model_svm = SVC(random_state=42)
model_svm.fit(X_resampled_train, y_resampled_train)
y_pred_svm = model_svm.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_svm))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_svm), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - SVM (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_svm):.4f}")

In [ ]:
# Huấn luyện mô hình Tabnet với dữ liệu đã qua resample
model_tabnet = TabNetClassifier(seed=42)

# đảm bảo inputs là numpy.ndarray (np.asarray hỗ trợ cả pandas và numpy)
X_tabnet_train = np.asarray(X_resampled_train)
y_tabnet_train = np.asarray(y_resampled_train)
X_tabnet_test = np.asarray(X_test)

model_tabnet.fit(X_tabnet_train, y_tabnet_train, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)

y_pred_tabnet = model_tabnet.predict(X_tabnet_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_tabnet))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_tabnet), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - TabNet (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_tabnet):.4f}")

Xây dựng mô hình với bộ dữ liệu đã qua resample bằng phương pháp SMOTE và loại bỏ outliers

In [ ]:
# Dùng data đã qua resample và loại bỏ outliers để train model
model = LogisticRegression(max_iter=1000)
model.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred = model.predict(X_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_cleaned_outliers_test, y_pred), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Logistic Regression (resampled + outliers removed)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred):.4f}")

In [ ]:
# Dùng data đã qua resample và clean outliers để train mô hình decision tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)

y_pred_dt = model_dt.predict(X_cleaned_outliers_test)

print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred_dt))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_cleaned_outliers_test, y_pred_dt), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Decision Tree (resampled + outliers removed)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_dt):.4f}")

In [ ]:
# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu đã qua resample và loại bỏ outliers
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_rf = model_rf.predict(X_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred_rf))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_cleaned_outliers_test, y_pred_rf), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - RandomForest (resampled + outliers removed)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_rf):.4f}")

In [ ]:
# Huan luyện mô hình KNN với dữ liệu đã qua resample và loại bỏ outliers cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
    y_pred_knn = model_knn.predict(X_cleaned_outliers_test)
    accuracy = accuracy_score(y_cleaned_outliers_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)   
model_knn.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_knn = model_knn.predict(X_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred_knn))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_cleaned_outliers_test, y_pred_knn), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - KNN (resampled + outliers removed)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_knn):.4f}")

In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu đã qua resample và loại bỏ outliers
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_xgb = model_xgb.predict(X_cleaned_outliers_test)

print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred_xgb))
disp = ConfusionMatrixDisplay(confusion_matrix(y_cleaned_outliers_test, y_pred_xgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - XGBoost (resampled + outliers removed)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM với dữ liệu đã qua resample và loại bỏ outliers
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_lgb = model_lgb.predict(X_cleaned_outliers_test)
print("Báo cáo phân loại:") 
print(classification_report(y_cleaned_outliers_test, y_pred_lgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_cleaned_outliers_test, y_pred_lgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_lgb):.4f}")

In [ ]:
# Huấn luyện mô hình SVM với dữ liệu đã qua resample và loại bỏ outliers
model_svm = SVC(random_state=42)
model_svm.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_svm = model_svm.predict(X_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_cleaned_outliers_test, y_pred_svm))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_cleaned_outliers_test, y_pred_svm))
print(f"Độ chính xác của mô hình: {accuracy_score(y_cleaned_outliers_test, y_pred_svm):.4f}")

In [ ]:
# Huấn luyện mô hình Tabnet với dữ liệu đã qua resample
model_tabnet = TabNetClassifier(seed=42)

# đảm bảo inputs là numpy.ndarray (hỗ trợ cả pandas và numpy)
X_tabnet_train = np.asarray(X_resampled_train)
y_tabnet_train = np.asarray(y_resampled_train)
X_tabnet_test = np.asarray(X_test)

model_tabnet.fit(X_tabnet_train, y_tabnet_train, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)

y_pred_tabnet = model_tabnet.predict(X_tabnet_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_tabnet))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_tabnet), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - TabNet (resampled)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_tabnet):.4f}")

In [ ]:
# Dùng tensorflow để xây dựng mô hình ANN (Artificial Neural Network)
model_ann = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_resampled_top12_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_ann.fit(X_resampled_top12_train, y_resampled_top12_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
y_pred_ann = (model_ann.predict(X_top_12_test) > 0.5).astype("int32")
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_ann))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_top_12_test, y_pred_ann))
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_ann):.4f}")


In [ ]:
# Dùng tensorflow để xây dựng mô hình ANN (Artificial Neural Network)
model_ann = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_resampled_top12_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_ann.fit(X_resampled_top12_train, y_resampled_top_12_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

y_pred_ann = (model_ann.predict(X_top_12_test) > 0.5).astype("int32")
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_ann))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_top_12_test, y_pred_ann))
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_ann):.4f}")


Huấn luyện mô hình với bộ dữ liệu chỉ lấy ra 12 đặc trưng quan trọng nhất

In [ ]:
# Huấn luyện các mô hình với bộ dữ liệu chỉ lấy ra 12 đặc trưng quan trọng nhất
# Huấn luyện mô hình Logistic Regression và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model = LogisticRegression(max_iter=1000)
model.fit(X_top_12_train, y_top_12_train)
y_pred = model.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Logistic Regression (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred):.4f}")

# Huấn luyện mô hình Decision Tree và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_top_12_train, y_top_12_train)
y_pred_dt = model_dt.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_dt))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_dt), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Decision Tree (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_dt):.4f}")

# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_top_12_train, y_top_12_train)
y_pred_rf = model_rf.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_rf))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_rf), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Random Forest (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_rf):.4f}")

# Huấn luyện mô hình KNN với dữ liệu chỉ có 12 đặc trưng quan trọng nhất cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_top_12_train, y_top_12_train)
    y_pred_knn = model_knn.predict(X_top_12_test)
    accuracy = accuracy_score(y_top_12_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị K tốt nhất: {best_k}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_top_12_train, y_top_12_train)
y_pred_knn = model_knn.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_knn))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_knn), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - KNN (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_knn):.4f}")

# Huấn luyện mô hình XGBoost với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_top_12_train, y_top_12_train)
y_pred_xgb = model_xgb.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_xgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_xgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - XGBoost (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_xgb):.4f}")
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_top_12_train, y_top_12_train)
y_pred_lgb = model_lgb.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_lgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_lgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - LightGBM (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_lgb):.4f}")

# Huấn luyện mô hình SVM với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_svm = SVC(random_state=42)
model_svm.fit(X_top_12_train, y_top_12_train)
y_pred_svm = model_svm.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_svm))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_svm), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - SVM (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_svm):.4f}")

# Huấn luyện mô hình Tabnet với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
X_tabnet = X_top_12_train if isinstance(X_top_12_train, np.ndarray) else X_top_12_train.values
y_tabnet = y_top_12_train if isinstance(y_top_12_train, np.ndarray) else y_top_12_train.values
model_tabnet = TabNetClassifier(seed=42)
model_tabnet.fit(X_tabnet, y_tabnet, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)
X_test_tabnet = X_top_12_test if isinstance(X_top_12_test, np.ndarray) else X_top_12_test.values
y_pred_tabnet = model_tabnet.predict(X_test_tabnet)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_tabnet))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_tabnet), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - TabNet (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_tabnet):.4f}")



Huấn luyện các mô hình với dữ liệu chỉ lấy 12 đặc trưng quan trọng nhất và có resampled bằng SMOTE


In [ ]:
# Huấn luyện các mô hình với bộ dữ liệu chỉ lấy ra 12 đặc trưng quan trọng nhất và resample bằng SMOTE
# Huấn luyện mô hình Logistic Regression và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model = LogisticRegression(max_iter=1000)
model.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred = model.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Logistic Regression (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred):.4f}")

# Huấn luyện mô hình Decision Tree và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred_dt = model_dt.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_dt))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_dt), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Decision Tree (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_dt):.4f}")

# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred_rf = model_rf.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_rf))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_rf), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Random Forest (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_rf):.4f}")

# Huấn luyện mô hình KNN với dữ liệu chỉ có 12 đặc trưng quan trọng nhất cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_resampled_top_12_train, y_resampled_top_12_train)
    y_pred_knn = model_knn.predict(X_top_12_test)
    accuracy = accuracy_score(y_top_12_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị K tốt nhất: {best_k}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred_knn = model_knn.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_knn))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_knn), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - KNN (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_knn):.4f}")

# Huấn luyện mô hình XGBoost với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred_xgb = model_xgb.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_xgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_xgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - XGBoost (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_xgb):.4f}")
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_top_12_train, y_top_12_train)
y_pred_lgb = model_lgb.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_lgb))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_lgb), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - LightGBM (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_lgb):.4f}")

# Huấn luyện mô hình SVM với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
model_svm = SVC(random_state=42)
model_svm.fit(X_resampled_top_12_train, y_resampled_top_12_train)
y_pred_svm = model_svm.predict(X_top_12_test)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_svm))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_svm), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - SVM (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_svm):.4f}")

# Huấn luyện mô hình Tabnet với dữ liệu chỉ có 12 đặc trưng quan trọng nhất
X_tabnet = X_resampled_top_12_train if isinstance(X_resampled_top_12_train, np.ndarray) else X_resampled_top_12_train.values
y_tabnet = y_resampled_top_12_train if isinstance(y_resampled_top_12_train, np.ndarray) else y_resampled_top_12_train.values
model_tabnet = TabNetClassifier(seed=42)
model_tabnet.fit(X_tabnet, y_tabnet, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)
X_test_tabnet = X_top_12_test if isinstance(X_top_12_test, np.ndarray) else X_top_12_test.values
y_pred_tabnet = model_tabnet.predict(X_test_tabnet)
print("Báo cáo phân loại:")
print(classification_report(y_top_12_test, y_pred_tabnet))
# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix(y_top_12_test, y_pred_tabnet), display_labels=[0,1])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix - TabNet (top 12 features)')
plt.show()
print(f"Độ chính xác của mô hình: {accuracy_score(y_top_12_test, y_pred_tabnet):.4f}")



# Tinh chỉnh các mô hình


In [ ]:

# Tinh chỉnh mô hình sử dụng GridSearchCV cho Random Forest và Logistic Regression
# GridSearch cho Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_rf.fit(X_resampled_train, y_resampled_train)
print(f"Best parameters for Random Forest: {grid_rf.best_params_}")
print(f"Best accuracy for Random Forest: {grid_rf.best_score_:.4f}")
best_rf = grid_rf.best_estimator_
from sklearn.metrics import f1_score, precision_recall_curve
# 1. Lấy xác suất dự đoán cho lớp '1' (nghỉ việc) trên tập TEST
y_pred_proba = best_rf.predict_proba(X_test)[:, 1]

# 2. Tìm F1-score cao nhất bằng cách thử nhiều ngưỡng
best_f1 = 0
best_threshold = 0
for threshold in np.arange(0.1, 0.6, 0.01): # Thử các ngưỡng từ 0.1 đến 0.5
    y_pred_custom = (y_pred_proba >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_custom, pos_label=1)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"Ngưỡng (threshold) tốt nhất: {best_threshold:.2f}")
print(f"F1-score cao nhất có thể đạt được: {best_f1:.4f}")

# 3. Đánh giá lại với ngưỡng tốt nhất
y_pred_final = (y_pred_proba >= best_threshold).astype(int)
print("\n--- Classification Report với Ngưỡng Tốt Nhất ---")
print(classification_report(y_test, y_pred_final))

# GridSearch cho Logistic Regression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}
grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=5, scoring='f1', n_jobs=-1)
grid_lr.fit(X_resampled_train, y_resampled_train)
print(f"Best parameters for Logistic Regression: {grid_lr.best_params_}")
print(f"Best accuracy for Logistic Regression: {grid_lr.best_score_:.4f}")

# Đánh giá lại trên tập kiểm tra
best_rf = grid_rf.best_estimator_
best_lr = grid_lr.best_estimator_

y_pred_rf_best = best_rf.predict(X_test)
y_pred_lr_best = best_lr.predict(X_test)

print("\nRandom Forest sau tinh chỉnh:")
print(classification_report(y_test, y_pred_rf_best))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred_rf_best):.4f}")

print("\nLogistic Regression sau tinh chỉnh:")
print(classification_report(y_test, y_pred_lr_best))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred_lr_best):.4f}")



Lưu lại các mô hình đã qua huấn luyện và các dataset qua các giai đoạn tiền xử lý khác nhau

In [ ]:
# Lưu mô hình đã huấn luyện sử dụng joblib
joblib.dump(best_rf, 'best_random_forest_model.pkl')
joblib.dump(best_lr, 'best_logistic_regression_model.pkl')
joblib.dump(model_knn, 'best_knn_model.pkl')
joblib.dump(model_dt, 'best_decision_tree_model.pkl')
joblib.dump(model_tabnet, 'best_tabnet_model.pkl')
joblib.dump(model_xgb, 'best_xgboost_model.pkl')
joblib.dump(model_lgb, 'best_lightgbm_model.pkl')
joblib.dump(model_svm, 'best_svm_model.pkl')
# Load mô hình và dự đoán lại để kiểm tra
loaded_rf = joblib.load('best_random_forest_model.pkl')
loaded_lr = joblib.load('best_logistic_regression_model.pkl')
loaded_knn = joblib.load('best_knn_model.pkl')
loaded_dt = joblib.load('best_decision_tree_model.pkl')
loaded_tabnet = joblib.load('best_tabnet_model.pkl')
loaded_xgb = joblib.load('best_xgboost_model.pkl')
loaded_lgb = joblib.load('best_lightgbm_model.pkl')
loaded_svm = joblib.load('best_svm_model.pkl')
